In [16]:
from IPython.display import display
import ipywidgets
import ipywidgets.widgets as widgets
import traitlets
from jetbot import Robot, Camera, bgr8_to_jpeg
from SCSCtrl import TTLServo

import threading
import cv2
import numpy as np
import paho.mqtt.client as mqtt
from datetime import datetime
import pytz
import time
import json
import random as rd

data published


In [17]:
robot = Robot()
#camera = Camera()
#cap = cv2.VideoCapture(0)

In [18]:
lbl1 = ipywidgets.Label(value="MQTT Test")

startBtn = widgets.Button(description='Start', button_style='info')

lbl2 = ipywidgets.Label(value="sensingData")
lbl3 = ipywidgets.Label(value="num1 : ")
num1lbl = ipywidgets.Label(value="0")
hbox1 = widgets.HBox([lbl3, num1lbl] )

lbl4 = ipywidgets.Label(value="num2 : ")
num2lbl = ipywidgets.Label(value="0")
hbox2 = widgets.HBox([lbl4, num2lbl] )
vbox1 = widgets.VBox([startBtn, lbl2, hbox1, hbox2])

manualBtn = widgets.Button(description='Manual', button_style='success', disabled = True)
lbl5 = ipywidgets.Label(value="commandData")
lbl6 = ipywidgets.Label(value="command : ")
commandlbl = ipywidgets.Label(value="None")
hbox3 = widgets.HBox([lbl6, commandlbl] )
vbox2 = widgets.VBox([manualBtn, lbl5, hbox3])

hbox4 = widgets.HBox([vbox1, vbox2] )

image_widget = ipywidgets.Image(format='jpeg', width=224, height=224)

display(lbl1, hbox4, image_widget)


#camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

data published
data published


Label(value='MQTT Test')

Image(value=b'', format='jpeg', height='224', width='224')

In [19]:
# 한국 시간대 (Asia/Seoul)로 설정
korea_timezone = pytz.timezone("Asia/Seoul")

#Broker IP Address 와 Port
#라즈베리파이5의 IP 주소로 수정필요
#address = "192.168.14.128"
address = "70.12.225.174"
port = 1883

cameraTopic = "AGV/camera"
commandTopic = "AGV/command"
sensingTopic = "AGV/sensing"
sensingData = {
    "time" : "None",
    "num1": 0.15,
    "num2": 0.99,
    "is_finish": 0,
    "manual_mode" : "off"
}
publishingData = None

In [34]:
xPos = 100
yPos = 0
servoPos_4 = 0


def agv_stop():
    robot.stop()
    
def agv_forward():
    robot.forward(0.4)

def agv_backward():
    robot.backward(0.4)

def agv_left():
    robot.left(0.3)

def agv_right():
    robot.right(0.3)
    
def limitCtl(maxInput, minInput, rawInput):
    if rawInput > maxInput:
        limitBuffer = maxInput
    elif rawInput < minInput:
        limitBuffer = minInput
    else:
        limitBuffer = rawInput
        
    return limitBuffer    

def cameraUp():
    global servoPos_5  # Global variables declaration
    servoPos_5 = limitCtl(25, -40, servoPos_5-15)
    TTLServo.servoAngleCtrl(5, servoPos_5, 1, 150)

def cameraDown():
    global servoPos_5  # Global variables declaration
    servoPos_5 = limitCtl(25, -40, servoPos_5+15)
    TTLServo.servoAngleCtrl(5, servoPos_5, 1, 150)

def camAngle(value):
    global servoPos_5
    servoPos_5 = limitCtl(25, -40, value)
    TTLServo.servoAngleCtrl(5, servoPos_5, 1, 150)
    
# Camera turn right
def ptRight():
    global servoPos_1
    servoPos_1 = limitCtl(80, -80, servoPos_1+15)
    TTLServo.servoAngleCtrl(1, servoPos_1, 1, 150)

# Camera turn left
def ptLeft():
    global servoPos_1
    servoPos_1 = limitCtl(80, -80, servoPos_1-15)
    TTLServo.servoAngleCtrl(1, servoPos_1, 1, 150)
    
def turnAngle(value):
    global servoPos_1
    servoPos_1 = limitCtl(80, -80, value)
    TTLServo.servoAngleCtrl(1, servoPos_1, 1, 150)
    
def grab(value):
    global servoPos_4
    servoPos_4 = value
    if servoPos_4 < -90:
        servoPos_4 = -90
    TTLServo.servoAngleCtrl(4, servoPos_4, 1, 150)
    
    
def arm1(value):
    global xPos
    xPos = value
    if xPos < 85:
        xPos = 85
#     if xPos < -150 or xPos > 150:
#         pass
    #TTLServo.xyInput(xPos, yPos)
    TTLServo.servoAngleCtrl(2, value, 1, 150)

def arm2(value):
    global yPos
    yPos = value
    if value <-50:
        value = -50

    #TTLServo.xyInput(xPos, yPos)
    TTLServo.servoAngleCtrl(3, value, 1, 150)
    

data published
data published
data published
{'time': '2024-11-12 13:58:23', 'cmd_string': 'arm1', 'arg_string': -7, 'is_finish': 1}{'time': '2024-11-12 13:58:23', 'cmd_string': 'arm1', 'arg_string': -7, 'is_finish': 1} <class 'dict'>
-7
 <class 'dict'>
{'time': '2024-11-12 13:58:23', 'cmd_string': 'arm1', 'arg_string': -11, 'is_finish': 1} <class 'dict'>
{'time': '2024-11-12 13:58:23', 'cmd_string': 'arm1', 'arg_string': -11, 'is_finish': 1} <class 'dict'>
-11
{'time': '2024-11-12 13:58:23', 'cmd_string': 'arm1', 'arg_string': -12, 'is_finish': 1} <class 'dict'>
-12
{'time': '2024-11-12 13:58:23', 'cmd_string': 'arm1', 'arg_string': -12, 'is_finish': 1} <class 'dict'>
{'time': '2024-11-12 13:58:23', 'cmd_string': 'arm1', 'arg_string': -28, 'is_finish': 1} <class 'dict'>
{'time': '2024-11-12 13:58:23', 'cmd_string': 'arm1', 'arg_string': -28, 'is_finish': 1} <class 'dict'>
-28
{'time': '2024-11-12 13:58:23', 'cmd_string': 'arm1', 'arg_string': -42, 'is_finish': 1} <class 'dict'>
{'time

In [26]:
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("connected OK")
    else:
        print("Bad connection Returned code=", rc)

def on_publish(client, userdata, result):
    print("data published")

def on_message(client, userdata, msg):
    global message
    message = json.loads(msg.payload.decode("utf-8"))
    print(message, type(message))
    
    commandlbl.value = message["cmd_string"]
    if message["cmd_string"] == "go":      agv_forward()
    elif message["cmd_string"] == "mid":   agv_stop()
    elif message["cmd_string"] == "left":  agv_left()
    elif message["cmd_string"] == "right": agv_right()
    elif message["cmd_string"] == "back":  agv_backward()
    elif message["cmd_string"] == "camera_angle" : 
        camAngle(message["arg_string"])
    elif message["cmd_string"] == "camera_turn_angle" : 
        turnAngle(message["arg_string"])
    elif message["cmd_string"] == "grab_angle" : 
        grab(message["arg_string"])
    elif message["cmd_string"] == "arm1" : 
        arm1(message["arg_string"])
        #print(message["arg_string"])
    elif message["cmd_string"] == "arm2" : 
        arm2(message["arg_string"])
        #print(message["arg_string"])
    elif message["cmd_string"] == "stop":  print('stop')
    elif message["cmd_string"] == "exit":
        print('exit')
        publishingData.stop()
        agv_stop()

data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
data published
{'time': '2024-11-12 13:52:35', 'cmd_string': 'arm1', 'arg_string': 2, 'is_finish': 1} <class 'dict'>
2{'time': '2024-11-12 13:52:35', 'cmd_string': 'arm1', 'arg_string': 2, 'is_finish': 1} <class 'dict'>

{'time': '2024-11-12 13:52:35', 'cmd_string': 'arm1', 'arg_string': 9, 'is_finish': 1} <class 'dict'>
9
{'time': '2024-11-12 13:52:35', 'cmd_string': 'arm1', 'arg_string': 9, 'is_finish': 1} <class 'dict'>
{'time': '2024-11-12 13:52:35', 'cmd_string': 'arm1', 'arg_string': 7, 'is_finish': 1}{'time': '2024-11-12 13:52:35', 'cmd_string': 'arm1', 'arg_string': 7, 'is_finish': 1} <class 'dict'>
 <class 'dict'>
7
{'time': '2024-11-12 13:52:35', 'cmd_string': 'arm1', 'arg_string': 2, 'is_finish': 1} <class 'dict'>
2
{'time': '2024-11-12 13:52:35', 'cmd_string': 'arm1',

In [22]:
class sensorReadPublish(threading.Thread):
    
    def __init__(self):
        super().__init__()
        self.th_flag = True
        
    def run(self):

        while self.th_flag:
		        #소수점 둘째자리까지 생성
            num1 = round(rd.random(),2)
            num2 = round(rd.random(),2)
            
            #Label widget 에 출력하기 위해 str 형변환
            num1lbl.value = str(num1)
            num2lbl.value = str(num2)
            
            #현재 시간 정보 가져오기
            current_time = datetime.now(korea_timezone)
            sensingData["time"] = current_time.strftime("%Y-%m-%d %H:%M:%S")
            sensingData["num1"] = num1
            sensingData["num2"] = num2
            
            #GUI Controller 로 publish 하
            client.publish(sensingTopic, json.dumps(sensingData), 1)
            time.sleep(0.5)
        
    def stop(self):
        self.th_flag = False


In [23]:
client = None
def startPub(change):
    global publishingData, startBtn, client
    
    if startBtn.description == "Start" :
		    #MQTT Client 객체 생성
        client = mqtt.Client()
        #Broker 연결
        client.connect(address, port)
        
        #Callback 함수 바인딩
        client.on_connect = on_connect
        client.on_publish = on_publish
        
        #loop_start() 를 이용해 비동기적으로 MQTT Protocol 동작
        client.loop_start()
        
        #sensorReadPublish() 객체 생성
        publishingData = sensorReadPublish()
				#thread 시작
        publishingData.start()
        
        #Start 버튼 정보 변경
        startBtn.description = "Stop"
        startBtn.button_style = 'danger'
        
        #Manual 버튼 활성화
        manualBtn.disabled = False
        
    elif startBtn.description == "Stop" :
    
		    #thread 종료
        publishingData.stop()
        
        startBtn.description = "Start"
        startBtn.button_style = 'info'
        #print("Stop", client.is_connected())

startBtn.on_click(startPub)

data published


In [24]:
def startSub(change):
    global manualBtn, client
    
    if manualBtn.description == "Manual" :
        #MQTT Subscribe 시작, topic, qos
        client.subscribe(commandTopic, 1)
        #on_message callback 함수 등록
        client.on_message = on_message
        
        manualBtn.description = "Auto"
        manualBtn.button_style = 'success'
        
    elif manualBtn.description == "Auto" :
        #MQTT Publish 종료
        publishingData.stop()
        manualBtn.description = "Manual"
        manualBtn.button_style = 'warning'
        manualBtn.disabled = True

manualBtn.on_click(startSub)

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "<ipython-input-14-e2b679a21cf7>", line 25, in run
    client.publish(sensingTopic, json.dumps(sensingData), 1)
AttributeError: 'NoneType' object has no attribute 'publish'

